## Read population data from folder population_data, join tables and save result as shapefile

#### Import libraries

In [172]:
from pyproj import CRS
import geopandas as gpd
from functools import reduce
import pandas as pd
import numpy as np

#### Create function to read population shape files and return population and geometry data in own dataframes

In [173]:
def read_population_data(year):
    
    # Define path to file
    fp = "population_data/"+str(year)+"/Vaestotietoruudukko_"+str(year)+".shp"
    
    # Read shapefile
    pop = gpd.read_file(fp)
    
    # Define coordinate reference system
    pop.crs = CRS.from_epsg(3879).to_wkt()
    
    # Drop columns if number of resudents is less than 0 (no value) or over 15000 (values without real location)
    pop = pop[(pop["ASUKKAITA"] > 0) & (pop["ASUKKAITA"] < 15000)]
    
    # Drop all the columns except index and asukkaita
    pop_pop = pop[["INDEX", "ASUKKAITA"]]
    pop_geom = pop[["INDEX", "geometry"]]
    
    # Print number of residents by year
    print(year, "number of residents:", pop_pop["ASUKKAITA"].sum())
    
    # Split data to two parts for later joining. First will have population data and second geometric data.
    
    # Rename column asukkaita to pop+year
    pop_pop = pop_pop.rename(columns={'ASUKKAITA': 'pop'+str(year)})
    
    # Rename column geometry to geometry+year. Must do to later fill missing values and merge tables.
    pop_geom = pop_geom.rename(columns={'geometry': 'geometry'+str(year)})
    
    # Set column INDEX to INDEX
    pop_pop = pop_pop.set_index("INDEX")
    pop_geom = pop_geom.set_index("INDEX")
    
    # Return result geodataframe
    return pop_pop, pop_geom

#### Read the files and save population and geometry data to own dataframes

##### Save to own files, because have to use different fillna methods.

In [174]:
# Data is available from years 1997-2003 and 2008-2019

# Years 1997 to 2003
pop1997_pop, pop1997_geom = read_population_data(1997)
pop1998_pop, pop1998_geom = read_population_data(1998)
pop1999_pop, pop1999_geom = read_population_data(1999)
pop2000_pop, pop2000_geom = read_population_data(2000)
pop2001_pop, pop2001_geom = read_population_data(2001)
pop2002_pop, pop2002_geom = read_population_data(2002)
pop2003_pop, pop2003_geom = read_population_data(2003)

# Years 2008 to 2012
pop2008_pop, pop2008_geom = read_population_data(2008)
pop2009_pop, pop2009_geom = read_population_data(2009)
pop2010_pop, pop2010_geom = read_population_data(2010)
pop2011_pop, pop2011_geom = read_population_data(2011)
pop2012_pop, pop2012_geom = read_population_data(2012)

# Years 2013 to 2019
pop2013_pop, pop2013_geom = read_population_data(2013)
pop2014_pop, pop2014_geom = read_population_data(2014)
pop2015_pop, pop2015_geom = read_population_data(2015)
pop2016_pop, pop2016_geom = read_population_data(2016)
pop2017_pop, pop2017_geom = read_population_data(2017)
pop2018_pop, pop2018_geom = read_population_data(2018)
pop2019_pop, pop2019_geom = read_population_data(2019)

1997 number of residents: 882280
1998 number of residents: 894264
1999 number of residents: 909602
2000 number of residents: 919561
2001 number of residents: 925631
2002 number of residents: 938930
2003 number of residents: 943607
2008 number of residents: 979739
2009 number of residents: 1007910
2010 number of residents: 1012326
2011 number of residents: 1025560
2012 number of residents: 1036528
2013 number of residents: 1049988
2014 number of residents: 1076765
2015 number of residents: 1087308
2016 number of residents: 1111940
2017 number of residents: 1121567
2018 number of residents: 1137053
2019 number of residents: 1155285


### Join tables to population from 1997 to 2012 and to population 2013 to 2019.

##### Hox! Between years 1997 and 2012 grid size 500m x 500m. Since 2013 grid size 250m x 250m. Thats why will create two different geodataframes and join them later.

#### First create dataframe with years 1997 to 2012

In [175]:
# Combine dataframes to lists
pop_list = [pop1997_pop, pop1998_pop, pop1999_pop, pop2000_pop, pop2001_pop, pop2002_pop, pop2003_pop,
           pop2008_pop, pop2009_pop, pop2010_pop, pop2011_pop, pop2012_pop]

geom_list = [pop1997_geom, pop1998_geom, pop1999_geom, pop2000_geom, pop2001_geom, pop2002_geom, pop2003_geom,
           pop2008_geom, pop2009_geom, pop2010_geom, pop2011_geom, pop2012_geom]

# Merge dataframes
pop = reduce(lambda left,right: pd.merge(left,right,on='INDEX', how="outer"), pop_list)
geom = reduce(lambda left,right: pd.merge(left,right,on='INDEX', how="outer"), geom_list)

# Fill missing population values with 0
pop = pop.fillna(0)

# Fill missing geometry value from left
geom["geometry"] = geom.apply(lambda x:  x[x.last_valid_index()], axis=1)
geom = geom["geometry"]

# Merge dataframes pop and geom
population = pop.join(geom)

# Convert dataframe back to geodataframe
population1997to2012 = gpd.GeoDataFrame(population)

# Set coordinate reference system
population1997to2012.crs = CRS.from_epsg(3879).to_wkt()

# Show 3 first row
population1997to2012.head(3)

,pop1997,pop1998,pop1999,pop2000,pop2001,pop2002,pop2003,pop2008,pop2009,pop2010,pop2011,pop2012,geometry
INDEX,,,,,,,,,,,,,
14,55.0,52.0,79.0,84.0,50.0,49.0,47.0,49.0,55.0,54.0,51.0,50.0,"POLYGON Z ((25488999.996 6695499.995 0.000, 25..."
16,25.0,29.0,20.0,23.0,33.0,35.0,34.0,38.0,38.0,38.0,36.0,32.0,"POLYGON Z ((25491000.002 6695499.995 0.000, 25..."
17,13.0,13.0,0.0,0.0,12.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((25492000.001 6695499.995 0.000, 25..."


#### Then create dataframe with years 2013 to 2019

In [176]:
# Combine dataframes to lists
pop_list = [pop2013_pop, pop2014_pop, pop2015_pop, pop2016_pop, pop2017_pop, pop2018_pop, pop2019_pop]
geom_list = [pop2013_geom, pop2014_geom, pop2015_geom, pop2016_geom, pop2017_geom, pop2018_geom, pop2019_geom]

# Merge dataframes
pop = reduce(lambda left,right: pd.merge(left,right,on='INDEX', how="outer"), pop_list)
geom = reduce(lambda left,right: pd.merge(left,right,on='INDEX', how="outer"), geom_list)

# Fill missing population values with 0
pop = pop.fillna(0)

# Fill missing geometry value from left
geom["geometry"] = geom.apply(lambda x:  x[x.last_valid_index()], axis=1)
geom = geom["geometry"]

# Merge dataframes pop and geom
population = pop.join(geom)

# Convert dataframe back to geodataframe
population2013to2019 = gpd.GeoDataFrame(population)

# Set coordinate reference system
population2013to2019.crs = CRS.from_epsg(3879).to_wkt()

# Show 3 first row
population2013to2019.head(3)

,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019,geometry
INDEX,,,,,,,,
688,5.0,8.0,8.0,7.0,9.0,9.0,0.0,"POLYGON Z ((25472499.995 6689749.005 0.000, 25..."
703,6.0,6.0,6.0,6.0,0.0,5.0,5.0,"POLYGON Z ((25472499.995 6685998.998 0.000, 25..."
710,7.0,8.0,8.0,8.0,8.0,8.0,8.0,"POLYGON Z ((25472499.995 6684249.004 0.000, 25..."


### Join tables population1997to2012 and population2013to2019

##### Now they have different geometries. Old one has 500m x 500m and new one 250m x 250m. Find centroid for new geometry and then find in which old grid the geometry belongs. After that join tables population 1997to2012 and population 2013to2019.

#### Find centroid of each geometry in population2013to2019

In [177]:
# Convert geometry to centroid of polygon
population2013to2019["geometry"] = population2013to2019.centroid

# Print first rows
population2013to2019.head(3)

,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019,geometry
INDEX,,,,,,,,
688,5.0,8.0,8.0,7.0,9.0,9.0,0.0,POINT (25472624.994 6689874.005)
703,6.0,6.0,6.0,6.0,0.0,5.0,5.0,POINT (25472624.994 6686123.998)
710,7.0,8.0,8.0,8.0,8.0,8.0,8.0,POINT (25472624.994 6684374.004)


#### Find old grid for each centroid in new dataframe

In [178]:
# Create column old geometry with grid system 500m x 500m
old_geometry = population1997to2012[["pop1997", "geometry"]]

# Spatial old geometry with population2013to2019, which had previously polygons 250m x 250m
pop2013to2019 = gpd.sjoin(old_geometry, population2013to2019, how="inner", op="contains")

# Drop unwanted column pop1997 and old index
pop2013to2019 = pop2013to2019.drop(['pop1997', 'index_right', 'geometry'], axis=1)

# Copy index to own column
pop2013to2019['index'] = pop2013to2019.index

# Group data by index and calculate sum of each index (population in old grid)
pop2013to2019 = pop2013to2019.groupby(pop2013to2019["index"]).sum()

#### Join dataframes population1997to2012 and population2013to2019 and change column order

In [179]:
# Join new dataframe to population1997to2012 dataframe
population = population1997to2012.merge(pop2013to2019, left_index=True, right_index=True, how='outer')

# Fill missing values with 0
population[["pop2013", "pop2014", "pop2015", "pop2016", "pop2017", "pop2017", "pop2018", "pop2019"]
          ] = population[["pop2013", "pop2014", "pop2015", "pop2016", "pop2017", "pop2017", "pop2018", "pop2019"]].fillna(0)

# Get columns names
columns = population.columns.tolist()

# Sort columns alphabetically
columns = sorted(columns)

# Set new order to dataframe
population = population[columns]

# Show first rows
population.head(3)

,geometry,pop1997,pop1998,pop1999,pop2000,pop2001,pop2002,pop2003,pop2008,pop2009,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019
14,"POLYGON Z ((25488999.996 6695499.995 0.000, 25...",55.0,52.0,79.0,84.0,50.0,49.0,47.0,49.0,55.0,54.0,51.0,50.0,50.0,49.0,53.0,56.0,54.0,52.0,51.0
16,"POLYGON Z ((25491000.002 6695499.995 0.000, 25...",25.0,29.0,20.0,23.0,33.0,35.0,34.0,38.0,38.0,38.0,36.0,32.0,29.0,25.0,28.0,26.0,26.0,26.0,25.0
17,"POLYGON Z ((25492000.001 6695499.995 0.000, 25...",13.0,13.0,0.0,0.0,12.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Add missing years 2004, 2005, 2006 and 2007

##### Lineary connect years 2003 and 2008. E.g. if pop2003=1 and pop2008=6, then pop2004=2, pop2005=3, pop2006=4 and pop2007=5.

In [180]:
# Reset index
population = population.reset_index(drop=True)

# Create function missing years to find population value
def missing_values(pop2003, pop2008, col):
    
    # If both values pop2003 and 2008 are 0, then return value 0
    if pop2008 == pop2003 == 0:
        return 0
    
    # If both have same value, then return that value
    if pop2008 == pop2003:
        return pop2008
    
    # Create values list, which has linear values between pop2003 and pop2008
    values = np.arange(pop2003, pop2008, (pop2008-pop2003)/4)
    
    # If year is 2004, then return first value in list
    if col == "pop2004":
        return values[0]
        
    # If year is 2005, then return second value in list
    elif col == "pop2005":
        return values[1]
        
    # If year is 2006, then return third value in list    
    elif col == "pop2006":
        return values[2]
        
    # If year is 2007, then return fourth value in list
    elif col == "pop2007":
        return values[3]
        
# Get population for years 2004 to 2007 using function missing_values        
for pop_year in ["pop2004", "pop2005", "pop2006", "pop2007"]:
    population[pop_year] = population.apply(lambda x : missing_values(x["pop2003"], x["pop2008"], pop_year), axis=1)

# Print population of years 2003 to 2008 and check how values look like
print(population[["pop2003", "pop2004", "pop2005", "pop2006", "pop2007", "pop2008"]])

      pop2003  pop2004  pop2005  pop2006  pop2007  pop2008
0        47.0     47.0    47.50     48.0    48.50     49.0
1        34.0     34.0    35.00     36.0    37.00     38.0
2        12.0     12.0     9.00      6.0     3.00      0.0
3         7.0      7.0     6.75      6.5     6.25      6.0
4         0.0      0.0     0.00      0.0     0.00      0.0
...       ...      ...      ...      ...      ...      ...
2385      5.0      5.0     6.00      7.0     8.00      9.0
2386     40.0     40.0    43.50     47.0    50.50     54.0
2387    579.0    579.0   569.75    560.5   551.25    542.0
2388    150.0    150.0   153.25    156.5   159.75    163.0
2389    140.0    140.0   137.75    135.5   133.25    131.0

[2390 rows x 6 columns]


### Change column order and population columns type to int

In [181]:
# Get columns names
columns = population.columns.tolist()

# Sort columns alphabetically
columns = sorted(columns)

# Set new order to dataframe
population = population[columns]

# Remove geometry from columns list
columns.remove("geometry")

# Loop through population columns and change type to int
for col in columns:
    population[col] = population[col].astype('int')

# Show first rows
population.head(3)  

,geometry,pop1997,pop1998,pop1999,pop2000,pop2001,pop2002,pop2003,pop2004,pop2005,...,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,pop2017,pop2018,pop2019
0,"POLYGON Z ((25488999.996 6695499.995 0.000, 25...",55,52,79,84,50,49,47,47,47,...,54,51,50,50,49,53,56,54,52,51
1,"POLYGON Z ((25491000.002 6695499.995 0.000, 25...",25,29,20,23,33,35,34,34,35,...,38,36,32,29,25,28,26,26,26,25
2,"POLYGON Z ((25492000.001 6695499.995 0.000, 25...",13,13,0,0,12,12,12,12,9,...,0,0,0,0,0,0,0,0,0,0


### Check that population of the years 2013 to 2019 has not changed in merge process

In [182]:
# Check that population sum has stayed same after convert to old grid system
for year in ["pop2013", "pop2014", "pop2015", "pop2016", "pop2017", "pop2018", "pop2019"]:
    print(year + " before merge: " + str(pop2013to2019[year].sum()) + 
      " After: " + str(population[year].sum()) + " Change: " + str(pop2013to2019[year].sum()-population[year].sum()))

pop2013 before merge: 1049700.0 After: 1049700 Change: 0.0
pop2014 before merge: 1075996.0 After: 1075996 Change: 0.0
pop2015 before merge: 1085593.0 After: 1085593 Change: 0.0
pop2016 before merge: 1109075.0 After: 1109075 Change: 0.0
pop2017 before merge: 1117231.0 After: 1117231 Change: 0.0
pop2018 before merge: 1131283.0 After: 1131283 Change: 0.0
pop2019 before merge: 1146766.0 After: 1146766 Change: 0.0


### Save dataframe population

In [183]:
# Save geodataframe
outfp = "population_data/population.shp"
population.to_file(outfp)   